### Training loops.

In this notebook we are going to create our own, trainning loops.

### Imports

In [32]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets
import numpy as np
import tensorflow_datasets as tfds

### Configuring the ``device`` for the environment.


In [3]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Let's create a model that will train on the `MNIST` dataset.

In [34]:
(ds_train, ds_test), ds_info = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [35]:
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

In [36]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64

ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_train.batch(128)
ds_test = ds_train.prefetch(AUTOTUNE)

 ### Let's create our custom trainning loop.

In [42]:
model = keras.Sequential([
      keras.layers.Flatten(input_shape=(28, 28)),
      keras.layers.Dense(64, activation="relu"),
      keras.layers.Dense(128, activation="relu"),
      keras.layers.Dense(10)
])

In [43]:
criterion = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam()
accuracy_metric = keras.metrics.SparseCategoricalAccuracy()

### The `train` loop.

In [44]:
epochs = 5
for epoch in range(epochs):
  print(f"Epoch: {epoch}/{epochs}")
  for X_batch, y_batch in ds_train:
    with tf.GradientTape() as tape:
      # forward pass
      y_pred = model(X_batch, training=True)
      loss = criterion(y_batch, y_pred)
    # calculate gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    
    # forward pass
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    #update the metrics
    accuracy_metric.update_state(y_batch, y_pred)
 
  print(f"loss: {loss:.3f}, accuracy: {accuracy_metric.result():.3f}")
  accuracy_metric.reset_state()

Epoch: 0/5
loss: 0.048, accuracy: 0.910
Epoch: 1/5
loss: 0.129, accuracy: 0.960
Epoch: 2/5
loss: 0.049, accuracy: 0.971
Epoch: 3/5
loss: 0.083, accuracy: 0.977
Epoch: 4/5
loss: 0.067, accuracy: 0.980


### The `test` loop.

In [45]:
for x_batch, y_batch in ds_test:
  y_pred = model(x_batch, training=False)
  loss = criterion(y_batch, y_pred)
  accuracy_metric.update_state(y_batch, y_pred)

print(f"loss: {loss:.3f}, accuracy: {accuracy_metric.result():.3f}")
accuracy_metric.reset_state()

loss: 0.084, accuracy: 0.985


That's how flexible are custom trainning loops.